## 1. 코사인 유사도

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

# 코사인 유사도 계산 함수
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))
    
doc1=np.array([0,1,1,1]) # 문서1
doc2=np.array([1,0,1,1]) # 문서2
doc3=np.array([2,0,2,2]) # 문서3

print("[코사인 유사도]")
print("문서1 <-> 문서2 :", cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print("문서1 <-> 문서3 :", cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print("문서2 <-> 문서3 :", cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

[코사인 유사도]
문서1 <-> 문서2 : 0.6666666666666667
문서1 <-> 문서3 : 0.6666666666666667
문서2 <-> 문서3 : 1.0000000000000002


> 문서1과 문서2의 코사인 유사도와 문서1과 문서3의 코사인 유사도가 같음.  

> 문서2와 문서3의 코사인 유사도가 1이 나옴.(유사도 값이 최대임)

## 2. 유사도를 이용한 추천 시스템 구현하기

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
data = pd.read_csv('../Data/movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
# title열(영화 제목)과 overview열(줄거리)을 코사인 유사도에 사용
# 20,000개의 샘플만 가지고 학습
data = data.head(20000)

# overview 결측치 확인
data['overview'].isnull().sum()

135

In [5]:
# overview에서 결측치를 가지는 경우 Null 값을 제거
data['overview'] = data['overview'].fillna('')

# 다시 overview 결측치 확인
data['overview'].isnull().sum()

0

In [6]:
# overview에 대해서 tf-idf 수행
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20000, 47487)


> 20,000개의 영화를 표현하기 위해 총 47,487개의 단어가 사용되었음

In [7]:
# 코사인 유사도를 사용하면 바로 문서의 유사도를 구함.
# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
# 영화의 타이틀과 인덱스를 가진 테이블 생성
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [9]:
# 영화의 타이틀을 입력하면 인덱스를 리턴
idx = indices['Father of the Bride Part II']
print(idx)

4


In [10]:
# 선택한 영화에 대해 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옴. 
    # 선택한 영화를 가지고 연산가능
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 계산.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옴.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['title'].iloc[movie_indices]

In [11]:
# 영화 '다크 나이트 라이즈'와 overview가 유사한 영화 상위 10개
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

> 가장 유사한 영화가 출력되는데, 영화 다크 나이트가 첫번째고, 그 외에도 전부 배트맨 영화임.